In [16]:
import pandas as pd
import pyomo.environ as pyomo

In [2]:
def read_excel(filename):
    """Read special Excel spreadsheet to input dict. 
    
    Args:
        filename: path to a spreadsheet file
        
    Returns
        dict of DataFrames, to be passed to create_model()
    """
    with pd.ExcelFile(filename) as xls:
        actor = xls.parse('Actor').set_index('Name')
        film = xls.parse('Film').set_index('Title')
        actor_film = xls.parse('Actor-Film').set_index(['Actor', 'Film'])
        
    data = {
        'actor': actor,
        'film': film,
        'actor-film': actor_film}

    return data

In [5]:
Datos = read_excel('input.xlsx')

In [8]:
Datos.keys()

dict_keys(['actor-film', 'film', 'actor'])

In [9]:
Datos["actor"]

,age,sink-source
Name,,
Lakeshia Westerberg,15,1
Rajab Heintze,27,0
Oceanus Yap,33,0
Tiger Probert,86,0
Kevin Bacon,39,-1


In [11]:
Datos["film"]

,genre,year
Title,,
The fast brat of Paris,Film noir,2001
The young fly,Action,2002
The red beard,Comedy,2003
Farm of ribbons,Drama,2004


In [10]:
Datos["actor-film"]

screen-time
Actor               Film                               
Lakeshia Westerberg The fast brat of Paris            5
Oceanus Yap         The fast brat of Paris           55
Lakeshia Westerberg The young fly                   100
Rajab Heintze       The young fly                    80
                    The red beard                    50
Oceanus Yap         The red beard                    23
Tiger Probert       The red beard                    11
                    Farm of ribbons                  45
Kevin Bacon         Farm of ribbons                  79

In [54]:
 """Create Pyomo ConcreteModel
    """
m = pyomo.ConcreteModel()
m.name = 'BACON'
m.data = Datos

In [55]:
# HELPER FUNCTIONS
def get_social_network(actor_film_table):
    social_network = []  # list of (actor, actor) pairs together in a film
    for film, group in actor_film_table.reset_index().groupby('Film'):
        all_actors = group['Actor'].unique()
        
        for a1, a2 in pd.MultiIndex.from_product([all_actors, all_actors]):
            if a1 != a2:  # only  include distinct
                social_network.append((a1, a2))
    return social_network

In [56]:

# pre-processing
social_network = get_social_network(m.data['actor-film'])

In [57]:
social_network

[('Tiger Probert', 'Kevin Bacon'),
 ('Kevin Bacon', 'Tiger Probert'),
 ('Lakeshia Westerberg', 'Oceanus Yap'),
 ('Oceanus Yap', 'Lakeshia Westerberg'),
 ('Rajab Heintze', 'Oceanus Yap'),
 ('Rajab Heintze', 'Tiger Probert'),
 ('Oceanus Yap', 'Rajab Heintze'),
 ('Oceanus Yap', 'Tiger Probert'),
 ('Tiger Probert', 'Rajab Heintze'),
 ('Tiger Probert', 'Oceanus Yap'),
 ('Lakeshia Westerberg', 'Rajab Heintze'),
 ('Rajab Heintze', 'Lakeshia Westerberg')]

In [58]:
# SETS
# elementary sets, e.g.
#    actor = {'Actor A', 'Actor B', 'Actor C'}
#    film = {'Film 1', 'Film 2', 'Film 3', 'Film 4'}
m.actor = pyomo.Set(initialize=m.data['actor'].index.unique())
m.film = pyomo.Set(initialize=m.data['film'].index.unique())

In [97]:
m.data['actor'].index.unique()

Index(['Lakeshia Westerberg', 'Rajab Heintze', 'Oceanus Yap', 'Tiger Probert',
       'Kevin Bacon'],
      dtype='object', name='Name')

In [59]:
# tuple sets, e.g.
#   plays_in = {('Actor A', 'Film 2'), ...}


m.plays_in = pyomo.Set(within=m.actor * m.film,
                       initialize=m.data['actor-film'].index)

In [60]:
a = m.actor * m.film

In [61]:
a.data()

{('Kevin Bacon', 'Farm of ribbons'),
 ('Kevin Bacon', 'The fast brat of Paris'),
 ('Kevin Bacon', 'The red beard'),
 ('Kevin Bacon', 'The young fly'),
 ('Lakeshia Westerberg', 'Farm of ribbons'),
 ('Lakeshia Westerberg', 'The fast brat of Paris'),
 ('Lakeshia Westerberg', 'The red beard'),
 ('Lakeshia Westerberg', 'The young fly'),
 ('Oceanus Yap', 'Farm of ribbons'),
 ('Oceanus Yap', 'The fast brat of Paris'),
 ('Oceanus Yap', 'The red beard'),
 ('Oceanus Yap', 'The young fly'),
 ('Rajab Heintze', 'Farm of ribbons'),
 ('Rajab Heintze', 'The fast brat of Paris'),
 ('Rajab Heintze', 'The red beard'),
 ('Rajab Heintze', 'The young fly'),
 ('Tiger Probert', 'Farm of ribbons'),
 ('Tiger Probert', 'The fast brat of Paris'),
 ('Tiger Probert', 'The red beard'),
 ('Tiger Probert', 'The young fly')}

In [62]:
#   plays_with = {('Actor A', 'Actor B'), ...}
m.plays_with = pyomo.Set(within=m.actor * m.actor,
                         initialize=social_network)

In [63]:
m.plays_with.data()

{('Kevin Bacon', 'Tiger Probert'),
 ('Lakeshia Westerberg', 'Oceanus Yap'),
 ('Lakeshia Westerberg', 'Rajab Heintze'),
 ('Oceanus Yap', 'Lakeshia Westerberg'),
 ('Oceanus Yap', 'Rajab Heintze'),
 ('Oceanus Yap', 'Tiger Probert'),
 ('Rajab Heintze', 'Lakeshia Westerberg'),
 ('Rajab Heintze', 'Oceanus Yap'),
 ('Rajab Heintze', 'Tiger Probert'),
 ('Tiger Probert', 'Kevin Bacon'),
 ('Tiger Probert', 'Oceanus Yap'),
 ('Tiger Probert', 'Rajab Heintze')}

In [64]:
# PARAMS
# not needed in this way of doing things. Instead, one can simply use the
# DataFrames directly within the constraint function. See fc_rule and 
# how it accesses the `sink-source` column from input.xlsx.

In [66]:
# VARIABLES
m.flow = pyomo.Var(m.actor, m.actor,  # domain (actor1, actor2)
                   within=pyomo.NonNegativeReals)  # flow >= 0

In [82]:
m.flow.get_values()

{('Kevin Bacon', 'Kevin Bacon'): None,
 ('Kevin Bacon', 'Lakeshia Westerberg'): None,
 ('Kevin Bacon', 'Oceanus Yap'): None,
 ('Kevin Bacon', 'Rajab Heintze'): None,
 ('Kevin Bacon', 'Tiger Probert'): None,
 ('Lakeshia Westerberg', 'Kevin Bacon'): None,
 ('Lakeshia Westerberg', 'Lakeshia Westerberg'): None,
 ('Lakeshia Westerberg', 'Oceanus Yap'): None,
 ('Lakeshia Westerberg', 'Rajab Heintze'): None,
 ('Lakeshia Westerberg', 'Tiger Probert'): None,
 ('Oceanus Yap', 'Kevin Bacon'): None,
 ('Oceanus Yap', 'Lakeshia Westerberg'): None,
 ('Oceanus Yap', 'Oceanus Yap'): None,
 ('Oceanus Yap', 'Rajab Heintze'): None,
 ('Oceanus Yap', 'Tiger Probert'): None,
 ('Rajab Heintze', 'Kevin Bacon'): None,
 ('Rajab Heintze', 'Lakeshia Westerberg'): None,
 ('Rajab Heintze', 'Oceanus Yap'): None,
 ('Rajab Heintze', 'Rajab Heintze'): None,
 ('Rajab Heintze', 'Tiger Probert'): None,
 ('Tiger Probert', 'Kevin Bacon'): None,
 ('Tiger Probert', 'Lakeshia Westerberg'): None,
 ('Tiger Probert', 'Oceanus Yap'

In [69]:
# OBJECTIVE
def obj_rule(m):
    return pyomo.summation(m.flow)  # == short for sum{a1,a2} flow[a1,a2]

In [70]:
# OBJECTIVE
m.obj = pyomo.Objective(rule=obj_rule,  # cf. `def obj_rule` below
                        sense=pyomo.minimize)  # or up or down?

In [72]:
pyomo.summation(m.flow).to_string()

flow[Lakeshia Westerberg,Lakeshia Westerberg] + flow[Lakeshia Westerberg,Oceanus Yap] + flow[Lakeshia Westerberg,Kevin Bacon] + flow[Lakeshia Westerberg,Tiger Probert] + flow[Lakeshia Westerberg,Rajab Heintze] + flow[Oceanus Yap,Lakeshia Westerberg] + flow[Oceanus Yap,Oceanus Yap] + flow[Oceanus Yap,Kevin Bacon] + flow[Oceanus Yap,Tiger Probert] + flow[Oceanus Yap,Rajab Heintze] + flow[Kevin Bacon,Lakeshia Westerberg] + flow[Kevin Bacon,Oceanus Yap] + flow[Kevin Bacon,Kevin Bacon] + flow[Kevin Bacon,Tiger Probert] + flow[Kevin Bacon,Rajab Heintze] + flow[Tiger Probert,Lakeshia Westerberg] + flow[Tiger Probert,Oceanus Yap] + flow[Tiger Probert,Kevin Bacon] + flow[Tiger Probert,Tiger Probert] + flow[Tiger Probert,Rajab Heintze] + flow[Rajab Heintze,Lakeshia Westerberg] + flow[Rajab Heintze,Oceanus Yap] + flow[Rajab Heintze,Kevin Bacon] + flow[Rajab Heintze,Tiger Probert] + flow[Rajab Heintze,Rajab Heintze]

In [76]:
m.plays_with.data()

{('Kevin Bacon', 'Tiger Probert'),
 ('Lakeshia Westerberg', 'Oceanus Yap'),
 ('Lakeshia Westerberg', 'Rajab Heintze'),
 ('Oceanus Yap', 'Lakeshia Westerberg'),
 ('Oceanus Yap', 'Rajab Heintze'),
 ('Oceanus Yap', 'Tiger Probert'),
 ('Rajab Heintze', 'Lakeshia Westerberg'),
 ('Rajab Heintze', 'Oceanus Yap'),
 ('Rajab Heintze', 'Tiger Probert'),
 ('Tiger Probert', 'Kevin Bacon'),
 ('Tiger Probert', 'Oceanus Yap'),
 ('Tiger Probert', 'Rajab Heintze')}

In [73]:
# CONSTRAINTS
def fc_rule(m, a):  # a == actor (from the `forall a in m.actor` line above)
    flow_balance = 0
    for (a1, a2) in m.plays_with:
        if a1 == a:
            flow_balance += m.flow[a1, a2]
        if a2 == a:
            flow_balance -= m.flow[a1, a2]
    
    # HA, finally an example for accessing a parameter value!
    flow_balance += m.data['actor'].loc[a, 'sink-source']
    
    return flow_balance == 0

In [74]:
# CONSTRAINTS
m.flow_conservation = pyomo.Constraint(m.actor,  # forall a in m.actor 
                                       rule=fc_rule)  # this must hold

In [87]:
m.flow_conservation.pprint()

flow_conservation : Size=5, Index=actor, Active=True
    Key                 : Lower : Body                                                                                                                                                                                                                             : Upper : Active
            Kevin Bacon :   0.0 :                                                                                                                                                           -1 - flow[Tiger Probert,Kevin Bacon] + flow[Kevin Bacon,Tiger Probert] :   0.0 :   True
    Lakeshia Westerberg :   0.0 :                                                            1 - flow[Oceanus Yap,Lakeshia Westerberg] + flow[Lakeshia Westerberg,Rajab Heintze] - flow[Rajab Heintze,Lakeshia Westerberg] + flow[Lakeshia Westerberg,Oceanus Yap] :   0.0 :   True
            Oceanus Yap :   0.0 :            flow[Oceanus Yap,Lakeshia Westerberg] - flow[Rajab Heintze,Oceanus Yap] - 

In [91]:
from pyomo.opt import SolverFactory

In [89]:
#prob = m.create_instance()


In [95]:
optim = SolverFactory('glpk')

result = optim.solve(m, tee=True)
#prob.load(result)
m.solutions.load_from(result)

# result statistics
print(result)


GLPSOL: GLPK LP/MIP Solver, v4.57
Parameter(s) specified in the command line:
 --write /var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmpbhgcrdhh.glpk.raw
 --wglp /var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmp5ar199m4.glpk.glp
 --cpxlp /var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmpqa21g5va.pyomo.lp
Reading problem data from '/var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmpqa21g5va.pyomo.lp'...
6 rows, 26 columns, 25 non-zeros
101 lines were read
Writing problem data to '/var/folders/jp/5g65spjj28z4bxpxm9h_hsjh0000gn/T/tmp5ar199m4.glpk.glp'...
88 lines were written
GLPK Simplex Optimizer, v4.57
6 rows, 26 columns, 25 non-zeros
Preprocessing...
5 rows, 12 columns, 24 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 4
      0: obj =   1.000000000e+00 inf =   1.000e+00 (1)
      1: obj =   3.000000000e+00 inf =   0.000e+00 (0)
*     2: obj 

In [96]:
# all variables
prob.display()

# custom print example
for actors in prob.flow:
    if prob.flow[actors].value > 0: 
        print("{} -> {}".format(actors[0], actors[1]))

Model BACON

  Variables:
    flow : Size=25, Index=flow_index, Domain=NonNegativeReals
        Key                                            : Lower : Value : Upper : Fixed : Stale
                        ('Kevin Bacon', 'Kevin Bacon') :     0 :   0.0 :  None : False : False
                ('Kevin Bacon', 'Lakeshia Westerberg') :     0 :   0.0 :  None : False : False
                        ('Kevin Bacon', 'Oceanus Yap') :     0 :   0.0 :  None : False : False
                      ('Kevin Bacon', 'Rajab Heintze') :     0 :   0.0 :  None : False : False
                      ('Kevin Bacon', 'Tiger Probert') :     0 :   1.0 :  None : False : False
                ('Lakeshia Westerberg', 'Kevin Bacon') :     0 :   0.0 :  None : False : False
        ('Lakeshia Westerberg', 'Lakeshia Westerberg') :     0 :   0.0 :  None : False : False
                ('Lakeshia Westerberg', 'Oceanus Yap') :     0 :   0.0 :  None : False : False
              ('Lakeshia Westerberg', 'Rajab Heintze') : 